In [1]:
import carla
import math
import random
import numpy as np
import cv2
import open3d as o3d
import subprocess
import time
import matplotlib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [4]:
vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [5]:
# Start the traffic light controller in the background
traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
print("Traffic light control is running in the background!")

# Start the traffic script in the background
traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '30', '--number-of-walkers', '10'])

print("Traffic generation is running in the background!")

time.sleep(5)

# Set up traffic manager
traffic_manager = client.get_trafficmanager()
vehicle.set_autopilot(True, traffic_manager.get_port())

Traffic light control is running in the background!
Traffic generation is running in the background!


In [6]:
# Load the 'plasma' colormap and convert it to a NumPy array
VIRIDIS = np.array(matplotlib.colormaps.get_cmap('plasma').colors)

# Generate a range of values between 0 and 1, matching the number of colors in VIRIDIS
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

# Generate a range of values for another colormap ('winter')
COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

# Load the 'winter' colormap and apply the generated range
COOL = np.array(matplotlib.colormaps.get_cmap('winter')(COOL_RANGE))

# Keep only the first three columns (RGB values), removing the alpha (transparency) channel
COOL = COOL[:, :3]

def add_open3d_axis(vis):
    """
    Add a small 3D axis to the Open3D visualizer.
    The axis consists of three colored lines:
    - Red for the X-axis
    - Green for the Y-axis
    - Blue for the Z-axis
    """

    # Create a LineSet object (used to represent lines in Open3D)
    axis = o3d.geometry.LineSet()

    # Define the 3D coordinates for the axis points:
    # The first point (0.0, 0.0, 0.0) is the origin
    # The second point (0.0, 0.0, 0.1) extends along the Z-axis (blue)
    # The third point (0.0, 0.1, 0.0) extends along the Y-axis (green)
    # The fourth point (0.1, 0.0, 0.0) extends along the X-axis (red)
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],  # Origin
        [0.0, 0.0, 0.1],  # Z-axis
        [0.0, 0.1, 0.0],  # Y-axis
        [0.1, 0.0, 0.0]   # X-axis
    ]))

    # Define the connections between the points to form lines:
    # The first line connects point 0 (origin) to point 1 (Z-axis)
    # The second line connects point 0 (origin) to point 2 (Y-axis)
    # The third line connects point 0 (origin) to point 3 (X-axis)
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],  # Z-axis (Blue)
        [0, 2],  # Y-axis (Green)
        [0, 3]   # X-axis (Red)
    ]))

    # Define the colors for each line in RGB format:
    # The first line (Z-axis) is blue
    # The second line (Y-axis) is green
    # The third line (X-axis) is red
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],  # Red (X-axis)
        [0.0, 1.0, 0.0],  # Green (Y-axis)
        [0.0, 0.0, 1.0]   # Blue (Z-axis)
    ]))

    # Add the axis geometry to the visualizer
    vis.add_geometry(axis)

In [7]:
def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity colors ready to be consumed by Open3D"""
    
    # Convert raw LiDAR data (binary buffer) into a NumPy array of 32-bit floats ('f4')
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    
    # Reshape the data into a 2D array where each row contains (x, y, z, intensity)
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Extract intensity values (last column)
    intensity = data[:, -1]
    
    # Normalize intensity values for color mapping using logarithmic scaling
    intensity_safe = np.clip(intensity, 1e-6, None)
    intensity_col = 1.0 - np.log(intensity_safe) / np.log(np.exp(-0.004 * 100))
    
    # Map intensity values to the VIRIDIS colormap for better visualization
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),  # Red channel
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),  # Green channel
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])   # Blue channel
    ]

    # Extract (x, y, z) spatial coordinates (exclude intensity)
    points = data[:, :-1]

    # Flip the x-axis to match Open3D coordinate conventions
    points[:, :1] = -points[:, :1]

    # Convert points and colors into Open3D format and update the point cloud list
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

def radar_callback(data, point_list):
    """Prepares radar data for visualization in Open3D"""
    
    # Initialize a NumPy array to store processed radar data (x, y, z, velocity)
    radar_data = np.zeros((len(data), 4))

    # Process each radar detection entry
    for i, detection in enumerate(data):
        # Convert polar coordinates (depth, altitude, azimuth) to Cartesian (x, y, z)
        x = detection.depth * math.cos(detection.altitude) * math.cos(detection.azimuth)
        y = detection.depth * math.cos(detection.altitude) * math.sin(detection.azimuth)
        z = detection.depth * math.sin(detection.altitude)

        # Store computed (x, y, z) coordinates along with velocity data
        radar_data[i, :] = [x, y, z, detection.velocity]

    # Extract the absolute velocity values to be used as intensity
    intensity = np.abs(radar_data[:, -1])

    # Normalize velocity values for color mapping using logarithmic scaling
    intensity_safe = np.clip(intensity, 1e-6, None)  # Replacing 0 with a small value
    intensity_col = 1.0 - np.log(intensity_safe) / np.log(np.exp(-0.004 * 100))

    
    # Map velocity-based intensity values to the COOL colormap for visualization
    int_color = np.c_[
        np.interp(intensity_col, COOL_RANGE, COOL[:, 0]),  # Red channel
        np.interp(intensity_col, COOL_RANGE, COOL[:, 1]),  # Green channel
        np.interp(intensity_col, COOL_RANGE, COOL[:, 2])   # Blue channel
    ]

    # Extract (x, y, z) spatial coordinates from radar data
    points = radar_data[:, :-1]

    # Flip the x-axis to match Open3D coordinate conventions
    points[:, :1] = -points[:, :1]

    # Convert processed radar data into Open3D format and update the point cloud list
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [8]:
# LIDAR SENSOR SETUP
# Find the LiDAR sensor blueprint
lidar_bp = bp_lib.find('sensor.lidar.ray_cast')

# Set LiDAR attributes (range, noise, field of view, channels, etc.)
lidar_bp.set_attribute('range', '100.0')               # Maximum sensing range in meters
lidar_bp.set_attribute('noise_stddev', '0.1')         # Noise standard deviation
lidar_bp.set_attribute('upper_fov', '15.0')           # Upper field of view angle
lidar_bp.set_attribute('lower_fov', '-25.0')          # Lower field of view angle
lidar_bp.set_attribute('channels', '64.0')            # Number of LiDAR channels
lidar_bp.set_attribute('rotation_frequency', '20.0')  # Rotations per second
lidar_bp.set_attribute('points_per_second', '500000') # Total points captured per second

# Define LiDAR's initial position relative to the vehicle
lidar_init_trans = carla.Transform(carla.Location(z=2))

# Spawn the LiDAR sensor and attach it to the vehicle
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)



# RADAR SENSOR SETUP
# Find the radar sensor blueprint
radar_bp = bp_lib.find('sensor.other.radar')

# Set Radar attributes (horizontal & vertical field of view, points per second)
radar_bp.set_attribute('horizontal_fov', '30.0')  # Horizontal field of view in degrees
radar_bp.set_attribute('vertical_fov', '30.0')    # Vertical field of view in degrees
radar_bp.set_attribute('points_per_second', '10000')  # Number of points per second

# Define Radar's initial position relative to the vehicle
radar_init_trans = carla.Transform(carla.Location(z=2))

# Spawn the Radar sensor and attach it to the vehicle
radar = world.spawn_actor(radar_bp, radar_init_trans, attach_to=vehicle)



# CAMERA SENSOR SETUP
# Find the RGB camera sensor blueprint
camera_bp = bp_lib.find('sensor.camera.rgb')

# Define Camera's initial position relative to the vehicle
camera_init_trans = carla.Transform(carla.Location(z=2.5, x=-3), carla.Rotation())

# Spawn the camera sensor and attach it to the vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)


# INITIALIZE POINT CLOUDS
# Create Open3D PointCloud objects for LiDAR and Radar
point_list = o3d.geometry.PointCloud()
radar_list = o3d.geometry.PointCloud()

# IMAGE DATA STORAGE
# Get camera image dimensions from blueprint attributes
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

# Create a dictionary to store image data as a NumPy array
camera_data = {'image': np.zeros((image_h, image_w, 4))}  # 4 channels (RGBA)

# SENSOR CALLBACK FUNCTIONS
# Assign callback functions to listen to sensor data streams
lidar.listen(lambda data: lidar_callback(data, point_list))  # LiDAR callback
radar.listen(lambda data: radar_callback(data, radar_list))  # Radar callback
camera.listen(lambda image: camera_callback(image, camera_data))  # Camera callback

In [ ]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)


# Create an Open3D Visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',  
    width=960,                  
    height=540,                 
    left=400,                  
    top=270                    
)

# Set Open3D rendering options
vis.get_render_option().background_color = [0.05, 0.05, 0.05]  # Dark background
vis.get_render_option().point_size = 1                         # Small point size
vis.get_render_option().show_coordinate_frame = True           # Show coordinate axes

# Add a small 3D axis to the Open3D window
add_open3d_axis(vis)

frame = 0  # Initialize frame counter

while True:
    if frame == 2:
        # Add point cloud data to the Open3D visualizer
        vis.add_geometry(point_list)  # LiDAR point cloud
        vis.add_geometry(radar_list)  # Radar point cloud

        # Update point cloud geometry
    vis.update_geometry(point_list)
    vis.update_geometry(radar_list)

    # Process user events and update the Open3D renderer
    vis.poll_events()
    vis.update_renderer()

    time.sleep(0.005)

    # Increment frame counter
    frame += 1

    # Update the camera feed in the OpenCV window
    cv2.imshow('RGB Camera', camera_data['image'])

    # If the user presses 'q', exit the visualization loop
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

radar.stop()
radar.destroy()

lidar.stop()
lidar.destroy()

camera.stop()
camera.destroy()

vis.destroy_window()